In [16]:
import numpy as np
import pandas as pd
import re
import datetime as dt

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import string

import nltk 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')
import matplotlib.patches as mpatches
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import StratifiedKFold
import joblib
import pickle

In [17]:
train = pd.read_csv('/Users/kimginam/Data176/NLP/소설작가분류/raw/train.csv')
test = pd.read_csv('/Users/kimginam/Data176/NLP/소설작가분류/raw/test_x.csv')

In [18]:
test.head(2)

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."


In [19]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [20]:
train.head(2)

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2


In [21]:
#Remove Urls and HTML links
def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)
train['text_new']=train['text'].apply(lambda x:remove_urls(x))
test['text_new']=test['text'].apply(lambda x:remove_urls(x))

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
train['text_new']=train['text_new'].apply(lambda x:remove_html(x))
test['text_new']=test['text_new'].apply(lambda x:remove_html(x))

In [22]:
# Lower casing
def lower(text):
    low_text= text.lower()
    return low_text
train['text_new']=train['text_new'].apply(lambda x:lower(x))
test['text_new']=test['text_new'].apply(lambda x:lower(x))


# Number removal
def remove_num(text):
    remove= re.sub(r'\d+', '', text)
    return remove
train['text_new']=train['text_new'].apply(lambda x:remove_num(x))
test['text_new']=test['text_new'].apply(lambda x:remove_num(x))

In [23]:
#Remove stopwords & Punctuations
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def punct_remove(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    return punct
train['text_new']=train['text_new'].apply(lambda x:punct_remove(x))
test['text_new']=test['text_new'].apply(lambda x:punct_remove(x))


def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
train['text_new']=train['text_new'].apply(lambda x:remove_stopwords(x))
test['text_new']=test['text_new'].apply(lambda x:remove_stopwords(x))

In [24]:
train.head(2)

,index,text,author,text_new
0,0,"He was almost choking. There was so much, so m...",3,almost choking much much wanted say strange ex...
1,1,"“Your sister asked for it, I suppose?”",2,sister asked suppose


In [25]:
test.head(2)

,index,text,text_new
0,0,“Not at all. I think she is one of the most ch...,think one charming young ladies ever met might...
1,1,"""No,"" replied he, with sudden consciousness, ""...",replied sudden consciousness find cannot ignor...


In [26]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X, tfidf_vectorizer = tfidf(train['text_new'])
X_tst = tfidf_vectorizer.transform(test['text_new'])

In [27]:
target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [28]:
y = train.author.values
y.shape

(54879,)

In [29]:
# joblib.dump(vec, 'TfidfVectorizer.pkl')

In [30]:
p = np.zeros((X.shape[0], n_class))
p_tst = np.zeros((X_tst.shape[0], n_class))
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
    clf = LogisticRegression()
    clf.fit(X[i_trn], y[i_trn])
    p[i_val, :] = clf.predict_proba(X[i_val])
    p_tst += clf.predict_proba(X_tst) / n_class

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [31]:
clf.score(X, y)

0.8093441935895334

In [32]:
# joblib.dump(clf, 'logisticregression.pkl')

In [33]:
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

Log Loss (CV):   0.8019
